This notebook is designed to analyze MIBI data by performing data preprocessing, splitting the dataset into training, validation, and test sets, and preparing it for model training.

The first step involves importing necessary libraries and modules, including polars for data manipulation and sklearn for dataset splitting.

The dataset is loaded from a CSV file, and relevant columns are selected for further analysis.

A GroupShuffleSplit is used to ensure that the data is split while maintaining the grouping of patients, which is crucial for avoiding data leakage.

The training, validation, and test sets are printed to provide an overview of the data distribution.



In [24]:
import polars as pl
from sklearn.model_selection import train_test_split,GroupShuffleSplit
from torchvision import transforms
import matplotlib.pyplot as plt
import numpy as np
import os


In [25]:
#project imports 
import sys
notebook_path=os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_path,'NN_Framework')))
from NN_Framework import mibi_dataset

In [26]:
full_cell_sheet=pl.read_csv(r"D:\MIBI-TOFF\Data_For_Amos\cleaned_expression_with_both_classification_prob_spatial_30_08_24.csv")
print(full_cell_sheet.columns)
cell_frame = full_cell_sheet[['label', 'fov','pred','Group','patient number']]
unique_fovs = cell_frame.unique(subset='fov')
print(unique_fovs.head(5))


['Unnamed: 0', 'cell_size', '128Te', '129Xe', '12C', '130Xe', '131Xe', '132Xe', '137Ba', '138Ba', '139La', '140Ce', '181Ta', '182Empty', '197Au', '23Na', '24Mg', '25Mg', '27Al', '28Si', '31P', '32Si', '39K', '40Ca', '41K', '56Fe', '64Zn', '66Zn', 'Alexa Fluor 488', 'Bax', 'CCR7', 'CD11c', 'CD14', 'CD163', 'CD20', 'CD206', 'CD21', 'CD3', 'CD31', 'CD4', 'CD45', 'CD45RA', 'CD45RO', 'CD56', 'CD68', 'CD69', 'CD8', 'COL1A1', 'DC-SIGN', 'Foxp3', 'Granzyme B', 'HLA-DR-DP-DQ', 'HLA-class-1-A-B-C', 'IDO-1', 'Ki67', 'LAG-3', 'MECA-79', 'MelanA', 'PD-1', 'S100A9-Calprotectin', 'SMA', 'SOX10', 'TCF1TCF7', 'TIM-3', 'Tox-Tox2', 'anti-Biotin', 'dsDNA', 'label', 'area', 'eccentricity', 'major_axis_length', 'minor_axis_length', 'perimeter', 'convex_area', 'equivalent_diameter', 'centroid-0', 'centroid-1', 'major_minor_axis_ratio', 'perim_square_over_area', 'major_axis_equiv_diam_ratio', 'convex_hull_resid', 'centroid_dif', 'num_concavities', 'fov', 'pred', 'pred_prob', 'class', 'score', 'spatial', 'Grou

In [27]:
train_val_splitter = GroupShuffleSplit(n_splits=1, test_size=0.20, random_state=1995)
train_idx, temp_idx = next(train_val_splitter.split(unique_fovs, groups=unique_fovs['patient number']))
train_data = unique_fovs[train_idx]

# Split the remaining data into validation and testing sets
val_test_splitter = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=1995)
val_idx, test_idx = next(val_test_splitter.split(unique_fovs[temp_idx], groups=unique_fovs[temp_idx]['patient number']))
val_data = unique_fovs[temp_idx][val_idx]
test_data = unique_fovs[temp_idx][test_idx]

print("Training Set:")
print(f"Length of training data: {len(train_data)}")
print(train_data.head(5),)
print("\nValidation Set:")
print(f"Length of training data: {len(val_data)}")
print(val_data.head(5))
print("\nTesting Set:")
print(f"Length of training data: {len(test_data)}")
print(test_data.head(5))


import pickle
from datetime import datetime

# Add date_time to the data
data_to_save = {
    'train_data': train_data.to_dict(),
    'val_data': val_data.to_dict(),
    'test_data': test_data.to_dict(),
    'date_time': datetime.now().isoformat()  # Adding current date and time
}

# Define the filename with the current date and time
filename = f'data_split_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pkl'

with open(filename, 'wb') as pickle_file:
    pickle.dump(data_to_save, pickle_file)

print(f"Data saved to '{filename}' with current date and time.")









Training Set:
Length of training data: 143
shape: (5, 5)
┌───────┬────────┬──────────────┬───────┬────────────────┐
│ label ┆ fov    ┆ pred         ┆ Group ┆ patient number │
│ ---   ┆ ---    ┆ ---          ┆ ---   ┆ ---            │
│ f64   ┆ str    ┆ str          ┆ str   ┆ i64            │
╞═══════╪════════╪══════════════╪═══════╪════════════════╡
│ 1.0   ┆ FOV10  ┆ CD4 T cell   ┆ G1    ┆ 3              │
│ 1.0   ┆ FOV124 ┆ B cell       ┆ G1    ┆ 12             │
│ 1.0   ┆ FOV174 ┆ tumor        ┆ G4    ┆ 107            │
│ 1.0   ┆ FOV382 ┆ CD4 APC      ┆ G4    ┆ 117            │
│ 1.0   ┆ FOV50  ┆ Unidentified ┆ G3    ┆ 73             │
└───────┴────────┴──────────────┴───────┴────────────────┘

Validation Set:
Length of training data: 17
shape: (5, 5)
┌───────┬────────┬───────────────┬───────┬────────────────┐
│ label ┆ fov    ┆ pred          ┆ Group ┆ patient number │
│ ---   ┆ ---    ┆ ---           ┆ ---   ┆ ---            │
│ f64   ┆ str    ┆ str           ┆ str   ┆ i64         

In [28]:
import pickle

# Load the data from the pickle file
with open(r'D:\MIBI-TOFF\scripts\Mibi-Analysis-py\data_split_20241027_202034.pkl', 'rb') as pickle_file:
    data_loaded = pickle.load(pickle_file)

train_data = pl.DataFrame(data_loaded['train_data'])
val_data = pl.DataFrame(data_loaded['val_data'])
test_data = pl.DataFrame(data_loaded['test_data'])

In [29]:

root_dir=data_path=r'D:\MIBI-TOFF\Data_For_Amos'
df=unique_fovs
prefix='FOV'
fov_col='fov'
label_col='Group'
image_paths=[]

# The loop moves through the predefined file structure of the MIBI dataset being used to identify relevent image samples while filtering out files that we do not want
#Theoretically provides 39 expressions but some patients have varying amounts
for fov_dir in os.listdir(root_dir):
            if fov_dir.startswith(prefix):#replace with Pathlib matching later
                fov_path = os.path.join(root_dir, fov_dir)
                
                if os.path.isdir(fov_path):
                    #print(fov_dir)
                    group = df.filter(pl.col(fov_col) == fov_dir)  #why oh why did I decide to decide to use polars instead of pandas for this test?
                    if group.height>0:
                        group_data = group[label_col].to_list()[0]  
                        #print(group_data,binarized_data)
                        tif_path = os.path.join(fov_path, 'TIFs')
                        if os.path.exists(tif_path):
                            # Load all .tiff files ignoring those with "segmentation in the name"
                            sublist=[]
                            for image_file in os.listdir(tif_path):
                                if (image_file.endswith('.tif') or image_file.endswith('.tiff')) and ('segmentation' not in image_file) and not (image_file[0].isdigit()):
                                    sublist.append(os.path.join(tif_path, image_file))
                            

                            sublist.sort()#Should normalize the data assuming the data format is maintained true for this dataset 
                            #should create a perminant mapping
                            image_paths.append(sublist)
                        num_sublists = len(sublist)



In [30]:
#Find the expressions that are not in every file should be equal to the number of fovs 177 in this sample
expressions={}


for fov in image_paths:
    for path in fov:
        filename = os.path.basename(path)
        if filename not in expressions:
            expressions[filename]=1
        else:
            expressions[filename]+=1
print(expressions)


{'Alexa Fluor 488.tif': 177, 'Bax.tif': 177, 'CCR7.tif': 177, 'CD11c.tif': 177, 'CD14.tif': 177, 'CD163.tif': 177, 'CD20.tif': 177, 'CD206.tif': 177, 'CD21.tif': 177, 'CD3.tif': 177, 'CD31.tif': 177, 'CD4.tif': 177, 'CD45.tif': 177, 'CD45RA.tif': 177, 'CD45RO.tif': 177, 'CD56.tif': 177, 'CD68.tif': 177, 'CD69.tif': 177, 'CD8.tif': 177, 'COL1A1.tif': 177, 'DC-SIGN.tif': 177, 'Foxp3.tif': 177, 'Granzyme B.tif': 177, 'HLA-DR-DP-DQ.tif': 177, 'HLA-class-1-A-B-C.tif': 177, 'IDO-1.tif': 177, 'Ki67.tif': 177, 'LAG-3.tif': 177, 'MECA-79.tif': 177, 'MelanA.tif': 177, 'PD-1.tif': 177, 'S100A9-Calprotectin.tif': 177, 'SMA.tif': 177, 'SOX10.tif': 177, 'TCF1TCF7.tif': 177, 'TIM-3.tif': 177, 'Tox-Tox2.tif': 177, 'anti-Biotin.tif': 177, 'dsDNA.tif': 177, 'dsDNA-enhanced.tif': 9, 'dsDNA_cont.tif': 6, 'mask_fov_164.tif': 1, 'CD4_smooth.tif': 3, 'Foxp3_smooth.tif': 1, 'CD14_smooth.tif': 1, 'CD31_smooth.tif': 1, 'CD8_smooth.tif': 2, 'MelanA_enhanced.tif': 1, 'MelanA_smooth.tif': 1, 'S100A9-Calprotectin_s

In [31]:
threshold=117
consistent_expressions=[key for key, value in expressions.items() if value > threshold]
#I have no idea why > is needed to make this work
print(len(consistent_expressions))
print(consistent_expressions)

39
['Alexa Fluor 488.tif', 'Bax.tif', 'CCR7.tif', 'CD11c.tif', 'CD14.tif', 'CD163.tif', 'CD20.tif', 'CD206.tif', 'CD21.tif', 'CD3.tif', 'CD31.tif', 'CD4.tif', 'CD45.tif', 'CD45RA.tif', 'CD45RO.tif', 'CD56.tif', 'CD68.tif', 'CD69.tif', 'CD8.tif', 'COL1A1.tif', 'DC-SIGN.tif', 'Foxp3.tif', 'Granzyme B.tif', 'HLA-DR-DP-DQ.tif', 'HLA-class-1-A-B-C.tif', 'IDO-1.tif', 'Ki67.tif', 'LAG-3.tif', 'MECA-79.tif', 'MelanA.tif', 'PD-1.tif', 'S100A9-Calprotectin.tif', 'SMA.tif', 'SOX10.tif', 'TCF1TCF7.tif', 'TIM-3.tif', 'Tox-Tox2.tif', 'anti-Biotin.tif', 'dsDNA.tif']


In [32]:

#Reduced lists of expressions
expression_types = ['MelanA.tif', 'Ki67.tif', 'SOX10.tif', 'COL1A1.tif', 'SMA.tif', 
                            'CD206.tif', 'CD8.tif', 'CD4.tif', 'CD45.tif', 'CD3.tif', 'CD20.tif', 'CD11c.tif']

In [33]:
expressions_test={}
for fov in image_paths:
    print(fov[0],len(fov))
    for path in fov:
        
        filename = os.path.basename(path)
        if filename not in expressions_test:
            expressions_test[filename]=1
        else:
            expressions_test[filename]+=1
len(image_paths)

D:\MIBI-TOFF\Data_For_Amos\FOV10\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV100\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV102\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV104\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV106\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV108\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV110\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV112\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV114\TIFs\Alexa Fluor 488.tif 40
D:\MIBI-TOFF\Data_For_Amos\FOV116\TIFs\Alexa Fluor 488.tif 40
D:\MIBI-TOFF\Data_For_Amos\FOV118\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV12\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV120\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV122\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV124\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TOFF\Data_For_Amos\FOV126\TIFs\Alexa Fluor 488.tif 39
D:\MIBI-TO

177

In [34]:
import os
from NN_Framework import mibi_data_prep
import importlib



In [35]:


output_path=r'D:\MIBI-TOFF\Scratch'

hdf5_path = os.path.join(output_path, 'training_512.h5') 

train_hdf5 = mibi_data_prep.fovs_to_hdf5(root_dir=data_path, df=train_data,patch_size_x=512, patch_size_y=512,hdf5_path=hdf5_path, prefix='FOV', fov_col='fov', label_col='Group', expressions=expression_types)

hdf5_path = os.path.join(output_path, 'validation_512.h5') 

val_hdf5 = mibi_data_prep.fovs_to_hdf5(root_dir=data_path, df=val_data,patch_size_x=512, patch_size_y=512,hdf5_path=hdf5_path, prefix='FOV', fov_col='fov', label_col='Group', expressions=expression_types)

hdf5_path = os.path.join(output_path, 'testing_512.h5') 
test_hdf5 = mibi_data_prep.fovs_to_hdf5(root_dir=data_path, df=test_data,patch_size_x=512, patch_size_y=512,hdf5_path=hdf5_path, prefix='FOV', fov_col='fov', label_col='Group', expressions=expression_types)


D:\MIBI-TOFF\Data_For_Amos\FOV120 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV122 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV14 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV156 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV158 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV16 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV160 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV162 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV164 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV166 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV168 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV170 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV18 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV182 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV186 is not in provided Dataframe
D:\MIBI-TOFF\Data_For_Amos\FOV188 is not in provided Dataf